# *Ramdom Forest With Pipeline*

In [112]:
from warnings import filterwarnings
filterwarnings('ignore')

In [113]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

%matplotlib inline

In [114]:
df = sns.load_dataset('tips')

In [115]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [116]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [117]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [118]:
df.corr(numeric_only=True)

,total_bill,tip,size
total_bill,1.000000,0.675734,0.598315
tip,0.675734,1.000000,0.489299
size,0.598315,0.489299,1.000000


In [119]:
df.time.unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [120]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df['time'])

In [121]:
df['time'].unique()

array([0, 1])

In [122]:
X = df.drop(['time'],axis=1)
y = df['time']

In [123]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [124]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: time, dtype: int32

In [125]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain,ytest = train_test_split(X,y, test_size=0.20, random_state=42)

In [126]:
Xtrain.shape, ytrain.shape

((195, 6), (195,))

In [127]:
Xtest.shape, ytest.shape

((49, 6), (49,))

In [128]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [129]:
numerical_cols = X.select_dtypes(include='number').columns
category_cols = X.select_dtypes(include='category').columns

In [130]:
numerical_cols

Index(['total_bill', 'tip', 'size'], dtype='object')

In [131]:
category_cols

Index(['sex', 'smoker', 'day'], dtype='object')

In [132]:
# feature Engineering Automation

# Numerical Pipelines--------------------------------------------------------------------------------------
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')), ##Missing Values
        ('scaler',StandardScaler()) ## feature Scaling
    ]

)

# Categorical Pipeline--------------------------------------------------------------------------------------
cat_pipeline=Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='most_frequent')), ## handling Missing values
                ('onehotencoder',OneHotEncoder()) ## Categorical features to numerical
                ]

            )


In [133]:
# Column Transformer--------------------------------------------------------------------------------------

preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, numerical_cols),
    ('cat_pipeline', cat_pipeline, category_cols)
])

In [134]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 Index(['total_bill', 'tip', 'size'], dtype='object')),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 Index(['sex', 'smoker', 'day'], dtype='object'))])

In [135]:
scaled_Xtrain = preprocessor.fit_transform(Xtrain)

In [136]:
scaled_Xtest = preprocessor.transform(Xtest)

# *`Model Traning Automation`*

In [137]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [138]:
models = {
    'Ranom Forest' : RandomForestClassifier(),
    'Logistic Regression' : LogisticRegression(),
    'Decision Tree':DecisionTreeClassifier()
}

In [139]:
def evaluate_model(Xtrain,ytrain,Xtest,ytest, models):
    
    report = {}
    
    for i in range(len(models)):
        model = list(models.values())[i]
        # Traning The Model 
        model.fit(Xtrain,ytrain)
        
        # Predicting the Model
        y_pred = model.predict(Xtest)
        
        # getting the Accuracy Score:
        
        model_score = accuracy_score(ytest,y_pred)
        
        report[list(models.keys())[i]] = model_score
        
    return report



In [140]:
evaluate_model(scaled_Xtrain,ytrain,scaled_Xtest,ytest,models)

{'Ranom Forest': 0.9591836734693877,
 'Logistic Regression': 1.0,
 'Decision Tree': 0.9387755102040817}

In [141]:
## Hypeparameter Tuning
params={'max_depth':[3,5,10,None],
            'n_estimators':[100,200,300],
            'criterion':['gini','entropy']
            }

In [142]:
classifer = RandomForestClassifier()

In [143]:
from sklearn.model_selection import RandomizedSearchCV

In [145]:
cv = RandomizedSearchCV(classifer, param_distributions=params, scoring='accuracy', cv=5, verbose=3)
cv.fit(scaled_Xtrain,ytrain)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.974 total time=   0.2s
[CV 2/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.923 total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=1.000 total time=   0.1s
[CV 4/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.949 total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.923 total time=   0.1s
[CV 1/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.923 total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=1.000 total time=   0.1s
[CV 4/5] END criterion=entropy, max_depth=10, n_estimators=200;, score=0.949 total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=10, n_estimators=200;, 

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [146]:
cv.best_params_

{'n_estimators': 200, 'max_depth': 10, 'criterion': 'entropy'}

In [148]:
y_pred = cv.predict(scaled_Xtest)

In [149]:
accuracy_score(ytest,y_pred)

0.9591836734693877